# Import Aggregated IV Dataframe and join Medical Data

In [4]:
# import relevant tools
import os
from datetime import datetime

import numpy as np
import pandas as pd
from faker import Faker

fake = Faker()

# Import fire and PM2.5 data

In [7]:
import sys

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
!pip install geopandas --quiet
    from google.colab import drive

    drive.mount("/content/drive")
    in_dir = (
        in_instrument
    ) = "/content/drive/MyDrive/capstone_fire/notebooks/instrument_2/"
    import drive.MyDrive.capstone_fire.modules.deep_ols as deep_ols
else:
    in_dir = in_instrument = "../data/instrument_2/"
%cd '..'
    import modules.deep_ols as deep_ols
    %cd 'notebooks'

/Users/jordan/Documents/GitHub/fire_capstone
/Users/jordan/Documents/GitHub/fire_capstone/notebooks


In [8]:
# read in data that contains fire and pm25
fd = pd.read_csv(in_instrument + "finalish_df_6.csv")
suicide_df = pd.read_csv(in_instrument + "Suicide.csv")
neurotic_df = pd.read_csv(in_instrument + "Neurotic Disorders.csv")
psychoses_df = pd.read_csv(in_instrument + "Affective Psychoses.csv")

In [9]:
def prep_med(df):
    df["ZCTA"] = df["patzip_year_month"].map(lambda x: x[:5])
    df["year"] = df["patzip_year_month"].map(lambda x: x[6:10])
    df["month"] = df["patzip_year_month"].map(
        lambda x: "0" + str(x[-1]) if int(x[-2:]) < 10 else x[-2:]
    )
    df["year_month"] = df.apply(lambda x: str(x["year"]) + str(x["month"]), axis=1)
    df = df.rename(columns={"number_of_visits": str(df.columns[2]).lower()})
    df = df.drop(list(df.columns[:3]) + ["year", "month"], axis=1)
    return df  # [['number_of_visits','year','month','year_month']]


sf = prep_med(suicide_df)
nf = prep_med(neurotic_df)
pf = prep_med(psychoses_df)

fd.ZCTA = fd.ZCTA.astype(str)
fd.year_month = fd.year_month.astype(str)

fd1 = fd.merge(sf, on=["ZCTA", "year_month"], how="left")
fd2 = fd1.merge(nf, on=["ZCTA", "year_month"], how="left")
df = fd2.merge(pf, on=["ZCTA", "year_month"], how="left")
df = df.drop_duplicates()

In [10]:
df.to_csv(in_instrument + "final_df_3.csv")

In [17]:
df

,Unnamed: 0,ZCTA,year_month,ins_1_no_bin_raw,ins_2_add_acres_raw,ins_3_norms,ins_4_add_acres_norms,ins_5_norm_bins_acres,ins_6_bins_raw,zip_pm25,...,total_population,percent_bach_deg_grad_new,percent_high_school_grad_new,percent_pop_age_15to19_new,percent_pop_female_age_15to19_new,percent_pop_male_age_10to14_new,percent_pop_male_age_15to19_new,suicide,neurotic disorders,affective psychoses
0,0,89010,199101,0.014313,2.171619,2.095820,0.001013,0.011318,24.261588,12.450976,...,275.6,14.272763,56.603842,13.526364,14.514991,3.99958,8.781446,NaN,NaN,NaN
1,1,89010,199102,0.003401,1.473353,0.499474,0.000688,0.007878,16.875655,11.255854,...,275.6,14.272763,56.603842,13.526364,14.514991,3.99958,8.781446,NaN,NaN,NaN
2,2,89010,199103,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.130000,...,275.6,14.272763,56.603842,13.526364,14.514991,3.99958,8.781446,NaN,NaN,NaN
3,3,89010,199104,0.024555,4.317860,3.633868,0.002025,0.024022,51.209927,7.899268,...,275.6,14.272763,56.603842,13.526364,14.514991,3.99958,8.781446,NaN,NaN,NaN
4,4,89010,199105,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.004146,...,275.6,14.272763,56.603842,13.526364,14.514991,3.99958,8.781446,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586358,548327,97635,201808,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.950000,...,138.0,8.472573,43.360501,2.356591,5.758929,12.83365,11.093300,NaN,NaN,NaN
586359,548328,97635,201809,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.200000,...,138.0,8.472573,43.360501,2.356591,5.758929,12.83365,11.093300,NaN,NaN,NaN
586360,548329,97635,201810,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.453125,...,138.0,8.472573,43.360501,2.356591,5.758929,12.83365,11.093300,NaN,NaN,NaN
586361,548330,97635,201811,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.415625,...,138.0,8.472573,43.360501,2.356591,5.758929,12.83365,11.093300,NaN,NaN,NaN


In [21]:
iv = pd.read_csv(in_instrument + "jnv1v2.csv")
iv.ZCTA = iv.ZCTA.astype(str)
iv.year_month = iv.year_month.astype(str)
iv

,Unnamed: 0,ZCTA,year_month,zip_pm25v1,zip_pm25v2
0,0,89010,199101,11.596532,11.544660
1,1,92356,199101,11.550569,12.053348
2,2,94592,199101,22.102599,21.597018
3,3,90290,199101,16.049131,16.336319
4,4,92705,199101,26.658671,26.892271
...,...,...,...,...,...
500483,500483,93527,201612,0.109802,1.579411
500484,500484,95552,201612,1.065107,1.383041
500485,500485,93528,201612,3.722142,4.696543
500486,500486,93460,201612,4.363550,5.247724


In [26]:
df2 = df.merge(iv, on=["ZCTA", "year_month"], how="left")
df2.to_csv(in_instrument+'final_iv.csv')